In [1]:
import os
import pickle
import itertools
import pandas as pd

from matplotlib import colors
import matplotlib.pyplot as plt

from map_engine.map_generator import map

from quest_engine.quest_generator import *
from quest_engine.freytags_fitness import freytags

from encounter_engine.world_encounters import *

from character_engine.character import *

In [2]:
nw = map()

nw.populate_map(n_locations=1024, name="terrain", relaxed=True, k=100)
nw.populate_map(n_locations=4096, name="civilisation", relaxed=True, k=250)
nw.populate_map(n_locations=36, name="story_act", relaxed=True, k=250)
nw.populate_map(n_locations=1024, name="threat", relaxed=True, k=100)

nw.attribute_view(seed_1=24, seed_2=34,
                  map_name="terrain", view_name="terrain")
nw.attribute_view(seed_1=98, seed_2=50, map_name="civilisation",
                  view_name="civilisation", double=False)
nw.attribute_view(seed_1=34, seed_2=34, map_name="threat",
                  view_name="threat", double=False)
nw.attribute_view(seed_1=16, seed_2=16, map_name="story_act",
                  view_name="story_act", double=False)

nw.attribute_centroids("civilisation")
nw.attribute_centroids("terrain")
nw.land_mask()

with open('logs/map_nw', 'wb') as f:
    pickle.dump(nw, f)

with open("logs/map_nw", "rb") as f:
    nw = pickle.load(f)

/Users/cetiners/Desktop/Thesis/human_error/map_engine/map_attributer.py:104: RuntimeWarning: invalid value encountered in divide
  average = sum_/count


Centroids attributed, attributes centroited
Centroids attributed, attributes centroited


In [3]:

muts = ["complete","single_point","random_point"]
xos = ["pmx","single_point","ar"]
mups =  [.2,.4,.6,.8]

for param in [p for p in itertools.product(*[muts,xos,mups])]:

            params = {
                "map"           :   nw,
                "mutation"      :   param[0],
                "xo"            :   param[1],
                "mu_p"          :   param[2],
                "gens"          :   500,
                "shelf_size"    :   200,
                "pop_size"      :   100,
                "print_it"      :   False,
                "brute_force"   :   False,
                "log"           :   True,
                "early_stop"    :   True,
            }

            ql = quest_library(params)

Found the required individual on gen 26
Found the required individual on gen 10
Found the required individual on gen 1
Found the required individual on gen 3
Found the required individual on gen 6
Found the required individual on gen 2
Found the required individual on gen 1
Found the required individual on gen 10
Found the required individual on gen 136
Found the required individual on gen 9
Found the required individual on gen 10
Found the required individual on gen 5
Found the required individual on gen 1
Found the required individual on gen 5
Found the required individual on gen 10
Found the required individual on gen 1
Found the required individual on gen 58
Found the required individual on gen 6
Found the required individual on gen 3
Found the required individual on gen 5
Found the required individual on gen 6
Found the required individual on gen 9
Found the required individual on gen 15
Found the required individual on gen 7
Found the required individual on gen 8
Found the requir

KeyboardInterrupt: 

In [4]:

path = "logs/"
files = os.listdir(path)
files = [f for f in files if f.endswith(".csv")]

df = pd.concat([pd.read_csv(path+f) for f in files])


for f in files:
    os.remove(path+f)
    
df.to_csv("logs/merged.csv", index=False)




ValueError: No objects to concatenate

In [11]:
!pip freeze /Users/cetiners/Desktop/gubudik.txt


zsh:1: /Users/cetiners/Desktop/Thesis/human_error/he_env/bin/pip: bad interpreter: /Users/cetiners/Desktop/Thesis/human_error/he_environment/bin/python3: no such file or directory
absl-py==0.11.0
affine==2.3.1
aiohttp==3.8.1
aiosignal==1.2.0
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
alembic==1.7.7
algorithmx==2.0.3
anaconda==0.0.1.1
anaconda-client @ file:///opt/concourse/worker/volumes/live/ce6e20ef-c449-4fb2-5463-03f67f4a0b31/volume/anaconda-client_1635342572171/work
anaconda-navigator==2.1.2
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1637161053845/work
antlr4-python3-runtime==4.9.3
anyio @ file:///opt/concourse/worker/volumes/live/fdfc134d-03e4-4e6b-4eab-c131ac108813/volume/anyio_1644481717647/work/dist
appdirs==1.4.4
applaunchservices @ file:///Users/ktietz/demo/mc3/conda-bld/applaunchservices_1630511705208/work
appnope @ file:///opt/concourse/worker/volumes/live/5f13e5b3-5355-4541-5fc3-f08850c73cf9/volume/appnope_1606859448618/work
app